In [52]:
import re
from datetime import datetime
import numpy as np
from scipy.stats import norm

def parse_option_details(input_string):
    patterns = {
        'Identifier': r'^(.*?) Bid',
        'Bid': r'Bid\s(.*?)\s×',
        'Bid Size': r'Bid.*?×\s(\d+)',
        'Mark': r'Mark\s(.*?)\s',
        'High': r'High\s(.*?)\s',
        'Volume': r'Volume\s(\d+)',
        'Open Interest': r'Open Interest\s(\d+)',
        'Ask': r'Ask\s(.*?)\s×',
        'Ask Size': r'Ask.*?×\s(\d+)',
        'Previous Close': r'Previous Close\s(.*?)\s',
        'Low': r'Low\s(.*?)\s',
        'Implied Volatility': r'Implied Volatility\s(.*?)\%',
        'Delta': r'Delta\s(.*?)\s',
        'Gamma': r'Gamma\s(.*?)\s',
        'Theta': r'Theta\s(.*?)[\s$]',
        'Vega': r'Vega\s(.*?)\s',
        'Rho': r'Rho\s(.*?)$',
    }
    option_details = {}
    for key, pattern in patterns.items():
        match = re.search(pattern, input_string, re.DOTALL)
        if match:
            option_details[key] = match.group(1).strip()
    return option_details

def parse_growth_input(growth_input):
    match = re.search(r"\+\$(\d+\.\d+) \(\+(\d+\.\d+)%\)", growth_input)
    if match:
        increase_amount = float(match.group(1))
        percentage_increase = float(match.group(2))
        return increase_amount, percentage_increase
    return None, None

def calculate_stock_prices_from_growth(increase_amount, percentage_increase):
    percentage_increase_decimal = percentage_increase / 100
    original_price_close = increase_amount / percentage_increase_decimal
    stock_price_pre_market = original_price_close + increase_amount
    return original_price_close, stock_price_pre_market

def black_scholes_call(S, K, T, r, sigma):
    """
    S: spot price
    K: strike price
    T: time to maturity
    r: interest rate
    sigma: volatility of underlying asset
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))

    call = (S * norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * norm.cdf(d2, 0.0, 1.0))

    return call

# Inputs
# print("Please paste the option details and Greeks:")
details_input = input()
# details_input = "NET $112 Call 2/9 Bid $0.95 × 30 Mark $1.14 High $1.05 Volume 191 Open Interest 1 Ask $1.33 × 85 Previous Close $0.09 Low $0.11 Implied Volatility 457.35% The Greeks Delta 0.1437 Gamma 0.0135 Theta -3.1666 Vega 0.0083 Rho 0.0002"

# print("Please enter the overnight growth information:")
growth_input = input()
# growth_input = "+$21.13 (+23.38%)Overnight"

# Parsing
option_details = parse_option_details(details_input)
increase_amount, percentage_increase = parse_growth_input(growth_input)
original_price_close, stock_price_pre_market = calculate_stock_prices_from_growth(increase_amount, percentage_increase)

# Estimating the call option ask price using Black-Scholes
S = stock_price_pre_market
K = float(re.search(r'\$(\d+)', option_details['Identifier']).group(1))
r = 0.01  # risk-free rate
current_date = datetime.now()

# Parse expiry date and include the current year
expiry_date_str = option_details['Identifier'].split()[-1] + f'/{current_date.year}'
expiry_date = datetime.strptime(expiry_date_str, '%m/%d/%Y')

# If expiry date is still in the past, it means the option expires next year
if expiry_date < current_date:
    expiry_date = expiry_date.replace(year=expiry_date.year + 1)



T = max((expiry_date - current_date).days / 365, 1/365)  # at least one day to maturityr = 0.01  # Assumed value, replace with current risk-free rate
sigma = float(option_details['Implied Volatility']) / 100

# Debugging
print(f"S: {S}")
print(f"K: {K}")
print(f"T: {T}")
print(f"r: {r}")
print(f"sigma: {sigma}")


estimated_new_option_ask_price_black_scholes = black_scholes_call(S, K, T, r, sigma)

# Output
print("\nParsed Option Details and Greeks:")
print(f"Current date: {current_date}")
print(f"Expiry date: {expiry_date}")
for detail, value in option_details.items():
    print(f"{detail}: {value}")

print(f"\nStock Price at Market Close: ${original_price_close:.2f}")
print(f"Stock Price Right Before Market Opens: ${stock_price_pre_market:.2f}")
print(f"Percentage Increase: {percentage_increase:.2f}%")
print(f"Estimated Call Option Ask Price 5 minutes after Market Opens: ${estimated_new_option_ask_price_black_scholes:.2f}")


S: 111.50639007698888
K: 112.0
T: 1.0
r: 0.01
sigma: 4.5735

Parsed Option Details and Greeks:
Current date: 2024-02-09 15:09:35.111029
Expiry date: 2025-02-09 00:00:00
Identifier: NET $112 Call 2/9
Bid: $0.95
Bid Size: 30
Mark: $1.14
High: $1.05
Volume: 191
Open Interest: 1
Ask: $1.33
Ask Size: 85
Previous Close: $0.09
Low: $0.11
Implied Volatility: 457.35
Delta: 0.1437
Gamma: 0.0135
Theta: -3.1666
Vega: 0.0083
Rho: 0.0002

Stock Price at Market Close: $90.38
Stock Price Right Before Market Opens: $111.51
Percentage Increase: 23.38%
Estimated Call Option Ask Price 5 minutes after Market Opens: $109.04
